# Исследование надёжности заёмщиков

## Задача

Установить влияние семейного положения, количества детей, уровня дохода и цели получения кредита в банке на то, погасит ли клиент своей кредит в срок.

## Описание данных

Таблица **data** (статистика о платёжеспособности клиентов) со следующими столбцами:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Оглавление

### [Шаг 1. Открытие файла с данными и изучение общей информации](#read_data)

### [Шаг 2. Предобработка данных](#data_preprocessing)
- [Обработка пропусков](#data_gaps)
- [Замена типа данных](#data_type_replacement)
- [Обработка дубликатов](#duplicate_handling)
- [Лемматизация](#lemmatization)
- [Категоризация данных](#categorization)

### [Шаг 3. Ответы на вопросы](#answer_on_questions)
- [Зависимость между количеством детей и возвратом кредита в срок](#children_dependence)
- [Зависимость между семейным положением и возвратом кредита в срок](#marital_status_dependence)
- [Зависимость между уровнем дохода и возвратом кредита в срок](#income_level_dependence)
- [Влияние цели кредита на его возврат в срок](#purpose_dependence)

### [Шаг 4. Общий вывод](#general_conclusion)

## Ход работы

#### Импортируем необходимые для работы библиотеки.

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem

## <a id="read_data">Шаг 1. Открытие файла с данными и изучение общей информации</a>

#### Откроем файл с данными и сохраним их в переменной *solvency_data*.

In [2]:
solvency_data = pd.read_csv('/datasets/data.csv')

#### Изучим общую информацию.

In [3]:
solvency_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Посчитаем количество пропусков в каждом столбце.

In [4]:
solvency_data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
pd.DataFrame(round((solvency_data.isna().mean() * 100), 2)).style.background_gradient('coolwarm')

,0
children,0
days_employed,10.1
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


#### Посчитаем предварительное число дубликатов в таблице.

In [6]:
solvency_data.duplicated().sum()

54

#### Взглянем на первые 10 строк таблицы.

In [7]:
solvency_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


#### Найдём максимальное и минимальное значения у столбцов с количественными данными и просмотрим значения в столбцах с категориальными данными. Для индикаторных столбцов выполним обе операции.

In [8]:
solvency_data.min(numeric_only=True)

children               -1.000000
days_employed      -18388.949901
dob_years               0.000000
education_id            0.000000
family_status_id        0.000000
debt                    0.000000
total_income        20667.263793
dtype: float64

In [9]:
solvency_data.max(numeric_only=True).round()

children                 20.0
days_employed        401755.0
dob_years                75.0
education_id              4.0
family_status_id          4.0
debt                      1.0
total_income        2265604.0
dtype: float64

In [10]:
solvency_data.query('children == 20 and gender == "F"').shape[0]

47

In [11]:
int(np.floor(solvency_data['days_employed'].max() / 365))

1100

In [12]:
solvency_data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [13]:
solvency_data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [14]:
solvency_data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [15]:
solvency_data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [16]:
solvency_data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [17]:
solvency_data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [18]:
solvency_data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [19]:
len(solvency_data['purpose'].unique())

38

In [20]:
solvency_data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

### Вывод

Датафрейм состоит из 21 525 строк и 12 столбцов. В столбцах *days_employed* и *income_type* имеются пропуски, причем их количество совпадает — по 2174 пропуска в каждом (около 10% данных). Пропущенные данные являются количественными и имеют тип *float64*.

Ручным поиском было найдено 54 дубликата.

Также в глаза бросаются следующие артефакты:
- Имена столбцов датафрейма неочевидно отражают суть параметров;
- В столбце *days_employed* имеются отрицательные значения;
- Данные в столбце *education* представлены в различных регистрах;
- Некоторые данные в столбце *purpose* одинаковы по смыслу, однако представлены неэквивалентными строками;
- В датафрейме есть клиенты с 20 детьми, причем 47 из них — женщины;
- В датафрейме есть клиент с -1 ребёнком;
- В датафрейме есть клиент с 1100 полными годами трудового стажа, а также клиенты с отрицательным стажем;
- В датафрейме есть клиент в возрасте 0 лет;
- В столбце *family_status* все данные записаны в нижнем регистре, кроме опции *Не женат / не замужем*;
- У одного из клиентов в столбце *gender* указано значение *XNA*, то есть не указан пол.

## <a id='data_preprocessing'>Шаг 2. Предобработка данных</a>

#### Дадим столбцам имена, наиболее ясно отражающие их смысл.

In [21]:
solvency_data.columns = ['children_amount', 'work_experience_days', 'age_years', 'education_level', \
           'education_level_id', 'marital_status', 'marital_status_id', 'sex', \
           'employment_type', 'past_debt', 'monthly_income', 'loan_purpose']

### <a id='data_gaps'>Обработка пропусков</a>

#### Посмотрим, есть ли связь между пропусками в стоблцах *work_experience_days* и *monthly_income*.

In [22]:
solvency_data.query('work_experience_days.isna() and monthly_income.isna()').shape[0] ==\
solvency_data.query('work_experience_days.isna()').shape[0]

True

Оказалось, что у каждого клиента с неизвестным общим трудовым стажем также неизвестен ежемесячный доход.

#### В столбце *work_experience_days* замечены отрицательные значения. Посчитаем их количество.

In [23]:
solvency_data.query('work_experience_days < 0').shape[0]

15906

15906 отрицательных значений из общего числа в 19351. Вероятно, произошла ошибка при вводе данных. Перед обработкой пропусков в этом столбце необходимо привести данные в порядок. Общий трудовой стаж в днях не может быть отрицательным числом.

#### Применим метод *abs()*, возвращающий модуль числа, к столбцу *work_experience_days*.

In [24]:
solvency_data['work_experience_days'] = solvency_data['work_experience_days'].apply(abs)

#### Необходимо также разобраться с людьми, чей трудовой стаж превышает собственный возраст.

In [25]:
solvency_data.query('work_experience_days / 365 > age_years').shape[0]

3519

Таких клиентов оказалось немало. Выясним типы занятости этих людей.

In [26]:
solvency_data.query('work_experience_days / 365 > age_years')['employment_type'].value_counts()

пенсионер      3443
сотрудник        50
компаньон        18
госслужащий       6
безработный       2
Name: employment_type, dtype: int64

Большинство из них оказалось пенсионерами. 3443 из 3856 пенсионеров в датафрейме совершили ошибку, если вводили данные самостоятельно, либо произошла ошибка системы.

Итак, пропуски имеются только столбцах *work_experience_days* и *monthly_income*. Они взаимосвязаны и количественные. Заменять пропущенные значения средним в данном случае некорректно, ибо у 3519 клиентов явно неверно задан трудовой стаж, а 3446 из них — пенсионеры.

 #### Заменим пропущенные значения в датафрейме медианными значениями, причём сделаем это с учётом типа занятости.

In [27]:
solvency_data['employment_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [28]:
employment_types = solvency_data['employment_type'].unique()

for employment_type in employment_types:
    solvency_data.loc[solvency_data['employment_type'] == employment_type] =\
    solvency_data.loc[solvency_data['employment_type'] == employment_type]\
    .fillna(solvency_data.loc[solvency_data['employment_type'] == employment_type].median())

In [29]:
solvency_data.isna().sum()

children_amount         0
work_experience_days    0
age_years               0
education_level         0
education_level_id      0
marital_status          0
marital_status_id       0
sex                     0
employment_type         0
past_debt               0
monthly_income          0
loan_purpose            0
dtype: int64

### Вывод

Пропущенные значения в столбцах *work_experience_days* и *monthly_income* оказались взаимосвязанными. В результате некоторого сбоя в столбце *work_experience_days* с заведомо неотрицательными значениями появились отрицательные. Также у некоторых клиентов (преимущественно пенсионеров) в этом столбце присутствуют значения, превышающие допустимый максимум (соответствующее значение в столбце *age_years*, умноженное на 365). Отрицательные значения были заменены на значения с противоположным знаком. Пропущенные значения были заменены медианными с учётом типа занятости.

### <a id='data_type_replacement'>Замена типа данных</a>

#### Так как общий трудовой стаж высчитывается в днях, а ежемесячный доход — в целочисленных условных единицах, приведём данные в столбцах *work_experience_days* и *monthly_income* к целочисленному типу *int64*.

In [30]:
solvency_data.loc[:, ['work_experience_days', 'monthly_income']] =\
solvency_data.loc[:, ['work_experience_days', 'monthly_income']].astype('int')

In [31]:
solvency_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children_amount         21525 non-null int64
work_experience_days    21525 non-null int64
age_years               21525 non-null int64
education_level         21525 non-null object
education_level_id      21525 non-null int64
marital_status          21525 non-null object
marital_status_id       21525 non-null int64
sex                     21525 non-null object
employment_type         21525 non-null object
past_debt               21525 non-null int64
monthly_income          21525 non-null int64
loan_purpose            21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Данные в столбцах *work_experience_days* и *monthly_income* были представлены в датафрейме значениями с плавающей запятой (типа *float64*). Они были приведены к целочисленному типу *int64*.

### <a id='duplicate_handling'>Обработка дубликатов</a>

Приведём данные в состояние, удобное для поиска дубликатов. На первом шаге было замечено, что значения в столбцах *education_level* и *marital_status* представлены в различных регистрах.

#### Приведём все строковые значения в столбцах *education_level* и *marital_status* к нижнему регистру.

In [32]:
solvency_data['education_level'] = solvency_data['education_level'].str.lower()
solvency_data['marital_status'] = solvency_data['marital_status'].str.lower()

In [33]:
solvency_data['education_level'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_level, dtype: int64

In [34]:
solvency_data['marital_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: marital_status, dtype: int64

#### Посчитаем число дубликатов с учётом регистра.

In [35]:
solvency_data.duplicated().sum()

71

С учётом регистра получаем уже не 54, а 71 дубликат. Возможная причина появления дубликатов — человеческий фактор или ошибка соединения с сервером при вводе данных, в результате чего многократное нажатие кнопки отправки данных могло привести к образованию дубликатов.

#### Избавимся от дубликатов.

In [36]:
solvency_data = solvency_data.drop_duplicates().reset_index(drop=True)
solvency_data.duplicated().sum()

0

### Вывод

Категориальные данные в столбцах *education_level* и *marital_status* датафрейма были приведены к нижнему регистру, благодаря чему был выявлен 71 дубликат. Все дубликаты были удалены.

### <a id='lemmatization'>Лемматизация</a>

Как было сказано выше, некоторые данные в столбце *loan_purpose* одинаковы по смыслу, однако представлены неэквивалентными строками. Чтобы привести их к общему виду, применим лемматизацию. На первом шаге было посчитано, что количество различных категорий в столбце *loan_purpose* — 38. Это не так много.

#### Просмотрим значения столбца *loan_purpose* вручную и выделим основные леммы.

In [37]:
solvency_data['loan_purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Оказалось, что по сути категорий всего 4:

1. свадьба
2. жилье/недвижимость
3. автомобиль
4. образование

In [38]:
loan_purposes = ['свадьба', 'жилье', 'недвижимость', 'автомобиль', 'образование']
m = Mystem()

#### Функция выполняет лемматизацию значения *value* и формирует список лемм *lemmas*, затем определяет, какая цель получения кредита из списка *loan_purposes* присутствует в списке лемм и меняет значение *value* на соответствующее значение *loan_purpose*. Цели *жилье* и *недвижимость* объединяются в единую цель *жилье/недвижимость*.

In [39]:
def change_loan_purpose(value):
    lemmas = m.lemmatize(value)
    for loan_purpose in loan_purposes:
        if loan_purpose in lemmas:
            value = loan_purpose
            if value == 'жилье' or value == 'недвижимость':
                value = 'жилье/недвижимость'
    return value

#### Применим функцию *change_loan_purpose()* к столбцу *loan_purpose* датафрейма и посмотрим на результат.

In [40]:
solvency_data['loan_purpose'] = solvency_data['loan_purpose'].apply(change_loan_purpose)

In [41]:
solvency_data['loan_purpose'].value_counts()

жилье/недвижимость    10811
автомобиль             4306
образование            4013
свадьба                2324
Name: loan_purpose, dtype: int64

### Вывод

В столбце *loan_purpose* обнаружено 38 разных по формулировке целей получения кредита. При их ручном просмотре выяснилось, что по сути категорий всего 4: свадьба, жилье/недвижимость, автомобиль и образование.

С помощью лемматизации 38 категорий сведены к 4 основным. Значения столбца *loan_purpose* были заменены.

### <a id='categorization'>Категоризация данных</a>

Выделим новые категории с учётом того, какие выводы нам предстоит делать в дальнейшем.

#### Разделим клиентов на 4 группы по уровню дохода.

In [42]:
pd.qcut(solvency_data['monthly_income'], 4).value_counts()

(107623.0, 142594.0]      5479
(195820.25, 2265604.0]    5364
(20666.999, 107623.0]     5364
(142594.0, 195820.25]     5247
Name: monthly_income, dtype: int64

#### Создадим столбец *monthly_income_interval*, где каждому клиенту будет соответствовать один из четырёх доходных интервалов.

In [43]:
solvency_data['monthly_income_interval'] =\
pd.cut(solvency_data['monthly_income'], [20666, 107623, 142594, 195820, 2265604])

#### Посмотрим на число клиентов с тем или иным количеством детей.

In [44]:
solvency_data['children_amount'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children_amount, dtype: int64

Нет так много различных варианов. Число клиентов с 4, 5 или некорректным числом детей достаточно мало. Разобьём клиентов на несколько категорий по числу детей.

#### Функция, получающая на вход число детей и возвращающая название одной из четырёх категорий:

1. нет детей
2. один ребенок
3. двое детей
4. трое или более детей

In [45]:
def make_children_status(value):
    if value <= 0:
        return 'нет детей'
    elif 0 < value <= 1:
        return 'один ребенок'
    elif 1 < value <= 2:
        return 'двое детей'
    else:
        return 'трое или более детей'

#### Создадим столбец *children_status* с результатом применения функции *make_children_status()* к столбцу *children_amount* датафрейма.

In [46]:
solvency_data['children_status'] = solvency_data['children_amount'].apply(make_children_status)

### Вывод

Проведена категоризация данных с целью использования категорий в дальнейшем исследовании.

Клиенты были отнесены к одной из четырёх категорий с соответствующим доходным интервалом и разделены на клинетов без детей, с одним ребёнком, двумя детьми, а также тремя или более детьми.

#### Посмотрим на таблицу, полученную после предобработки данных.

In [47]:
solvency_data.head(10)

,children_amount,work_experience_days,age_years,education_level,education_level_id,marital_status,marital_status_id,sex,employment_type,past_debt,monthly_income,loan_purpose,monthly_income_interval,children_status
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье/недвижимость,"(195820, 2265604]",один ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,"(107623, 142594]",один ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье/недвижимость,"(142594, 195820]",нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,"(195820, 2265604]",трое или более детей
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,"(142594, 195820]",нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,жилье/недвижимость,"(195820, 2265604]",нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,жилье/недвижимость,"(195820, 2265604]",нет детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"(107623, 142594]",нет детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,"(20666, 107623]",двое детей
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,жилье/недвижимость,"(142594, 195820]",нет детей


## <a id='answer_on_questions'>Шаг 3. Ответы на вопросы</a>

### <a id='children_dependence'>Зависимость между количеством детей и возвратом кредита в срок</a>

In [48]:
children_status_pivot = solvency_data.query('children_amount != -1 and children_amount != 20')\
.pivot_table(index='children_status', values='past_debt', aggfunc=['sum', 'count', 'mean'])
children_status_pivot.columns = ['with_debt', 'total', 'debtor_ratio']
children_status_pivot.sort_values(by='debtor_ratio')

,with_debt,total,debtor_ratio
children_status,,,
нет детей,1063,14091,0.075438
трое или более детей,31,380,0.081579
один ребенок,444,4808,0.092346
двое детей,194,2052,0.094542


### Вывод

Таблица показывает следующую картину:
- из 14091 клиента без детей 1063 имели задолженности по возврату кредитов, то есть около 7%;
- из 4808 клиентов с одним ребёнком 444 имели задолженности по возврату кредитов, то есть около 9%;
- из 2052 клиентов с двумя детьми 194 имели задолженности по возврату кредитов, то есть около 9%;
- из 380 клиентов с тремя или более детьми 31 имел задолженности по возврату кредитов, то есть около 8%.

Около 7-9% клиентов имели задолженности по возврату кредита. Клиенты без детей возвращают кредит в срок немного чаще.

### <a id='marital_status_dependence'>Зависимость между семейным положением и возвратом кредита в срок</a>

In [49]:
marital_status_pivot = solvency_data\
.pivot_table(index='marital_status', values='past_debt', aggfunc=['sum', 'count', 'mean'])
marital_status_pivot.columns = ['with_debt', 'total', 'debtor_ratio']
marital_status_pivot.sort_values(by='debtor_ratio')

,with_debt,total,debtor_ratio
marital_status,,,
вдовец / вдова,63,959,0.065693
в разводе,85,1195,0.071130
женат / замужем,931,12339,0.075452
гражданский брак,388,4151,0.093471
не женат / не замужем,274,2810,0.097509


### Вывод

Таблица показывает следующую картину:
- из 959 овдовевших клиентов 63 имели задолженности по возврату кредитов, то есть около 6%;
- из 1195 клиентов в разводе 85 имели задолженности по возврату кредитов, то есть около 7%;
- из 12339 клиентов в официальном браке 931 имел задолженности по возврату кредитов, то есть около 7%;
- из 4151 клиента в гражданском браке 388 имели задолженности по возврату кредитов, то есть около 9%;
- из 2810 клиентов, не состоящих в браке, 274 имели задолженности по возврату кредитов, то есть около 10%.

Клиенты, не состоящие в браке, немного реже возвращают кредит вовремя — каждый десятый клиент имел задолженности. Возможно, это связано с бытовой неопытностью, приведшей к спонтанному взятию кредита.

Охотнее всех отдают кредиты овдовевшие клиенты — всего 6% должников. Возможно, это уже не первый кредит для них, и такие клиенты уже имеют опыт получения кредита.

### <a id='income_level_dependence'>Зависимость между уровнем дохода и возвратом кредита в срок</a>

In [50]:
monthly_income_pivot = solvency_data\
.pivot_table(index='monthly_income_interval', values='past_debt', aggfunc=['sum', 'count', 'mean'])
monthly_income_pivot.columns = ['with_debt', 'total', 'debtor_ratio']
monthly_income_pivot.sort_values(by='debtor_ratio')

,with_debt,total,debtor_ratio
monthly_income_interval,,,
"(195820, 2265604]",383,5364,0.071402
"(20666, 107623]",427,5364,0.079605
"(142594, 195820]",448,5247,0.085382
"(107623, 142594]",483,5479,0.088155


### Вывод

Клиенты с высоким и низким доходами немного реже становятся должниками (7% и 8% должников соответственно). Возможно, это связано с тем, что богатым клиентам проще отдать долги вовремя ввиду их высокого дохода, а бедные клиенты с опаской берут кредиты и тщательно продумывают, хватит ли им средств погасить кредит вовремя.

### <a id='purpose_dependence'>Влияние цели кредита на его возврат в срок</a>

In [51]:
loan_purpose_pivot = solvency_data\
.pivot_table(index='loan_purpose', values='past_debt', aggfunc=['sum', 'count', 'mean'])
loan_purpose_pivot.columns = ['with_debt', 'total', 'debtor_ratio']
loan_purpose_pivot.sort_values(by='debtor_ratio')

,with_debt,total,debtor_ratio
loan_purpose,,,
жилье/недвижимость,782,10811,0.072334
свадьба,186,2324,0.080034
образование,370,4013,0.092200
автомобиль,403,4306,0.093590


### Вывод

Охотнее всех кредит отдают клиенты, целью которых была операция с недвижимостью — 7% должников. Чаще всего это взрослые люди, построившие себе карьеру и готовые проводить такие серьёзные денежные операции.

Чуть реже долги отдают клиенты, получившие кредит на свадьбу. Клиенты, взявшие кредит на образование или авто, отдают деньги в срок ещё реже — 9% должников в каждой из категорий. Образование нынче дорогое, особенно зарубежное. Возникает много непредвиденных обстоятельств. Автомобиль же, порой, приобретается как статусная роскошь, которая, как видим, не всегда себя оправдывает, превращая клиента в должника.

## <a id='general_conclusion'>Шаг 4. Общий вывод</a>

На вход получены данные о платёжеспособности клиентов банка. Оказалось, что они находятся в неудобном для анализа состоянии:

1. Названия столбцов таблицы неясно отражают их суть.
2. В данных имеются пропуски.
3. В данных имеется несоответствие типов.
4. В данных имеются дубликаты.
5. Часть данных некорректна:
  - отрицательные значения в заведомо неотрицательных столбцах;
  - огромные значения, превышающие логический максимум;
  - различное представление эквивалентных данных и т.п.

Столбцы датафрейма были переименованы. Пропуски в данных о трудовом стаже и ежемесячном доходе оказались взаимосвязаны, а значит, скорее всего, поступали из одного источника.
Стаж некоторых клиентов превышал их возраст. Оказалось, что большинство таких клиентов — пенсионеры, возможно, самостоятельно совершившие ошибку при вводе данных.
У большей части клиентов оказался отрицательный трудовой стаж. Скорее всего, в системе случился сбой. Такие значения были заменены на противоположные.
Пропуски были заменены на медианное значение с учётом типа занятости.

Трудовой стаж и ежемесячный доход в таблице были представлены вещественными числами. Эти значения были приведены к целочисленному типу.

Расхождения в регистре данных в столбце с уровнем дохода и семейным положением затрудняли обработку дубликатов. Эти значения были приведены к нижнему регистру, а все дубликаты удалены.

Оказалось, что клиенты указали 38 различных целей получения кредита. Все категории были просмотрены вручную. Оказалось, что на деле их всего 4 — клиенты брали кредит на покупку автомобиля, свадьбу, образование или операции с недвижимостью. Значения столбца с целью получения кредита были заменены на одну из вышеперечисленных целей с использованием лемматизации.

С учётом необходимости нахождения ответа на поставленные заказчиком вопросы, данные были также категоризированы.
Клиенты были разбиты на 4 группы по уровню дохода. Также отдельно были выделены клиенты без детей, с одним ребёнком, с двумя детьми, а также с тремя или более детьми.

##### Зависимость между количеством детей и возвратом кредита в срок:
Зависимости практически не наблюдается. Незначительно реже должниками становятся бездетные клиенты. Уровень клиентов, имевших задолженности, составил примерно 7-9%.
##### Зависимость между семейным положением и возвратом кредита в срок:
Клиенты, не состоящие в браке, отдают кредит в срок реже — 10% имели задолженности. Овдовевшие клиенты отдают кредит вовремя чаще — всего 6% должников.
##### Зависимость между уровнем дохода и возвратом кредита в срок:
Клиенты с низким и высоким доходами реже становятся должниками (8% и 7% должников соответственно), а клиенты со средним доходом немного чаще — около 9% должников.
##### Влияние цели кредита на его возврат в срок:
Среди клиентов, взявших кредит на недвижимость, всего 7% должников, тогда как клиенты, получившие кредит на автомобиль или образование, имели задолженности немного чаще — более 9% должников.